In [1]:
import Pkg; Pkg.activate(dirname(@__DIR__)); Pkg.instantiate()

  Activating environment at `C:\Users\Grey\Documents\CMU work\robotics\final_project\Project.toml`


In [2]:
using LinearAlgebra
using ForwardDiff
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Plots
using OrdinaryDiffEq
using FileIO
using JLD2
using Printf
plotlyjs()

WebIO._IJuliaInit()

In [114]:
w = 30 # distance to simulate
n = 100 # number of discrete segments

Δx = w/n # width of each discrete segment


0.3

In [133]:
function snell(l, n, v1, v2)
    r = v2 ./ v1
    c = -dot(n, l)
    if c < 0
        c = c .* -1
        n = n .* -1
    end
    radicand = 1 - r ^ 2 * (1 - c ^ 2)
    if radicand < 0
        # total internal reflection
        return l + 2 .* c .* n
    else
        # refraction
        return r .* l .+ (r * c - sqrt(radicand)) .* n
    end
end

struct Surface
    func   # implicit surface, func(x,y) == 0
    x1     # lower bound for x values that are actually on the surface
    x2     # upper bound for x values ...
    vLeft  # IOR on left of surface, v1(x,y)
    vRight # IOR on right of surface, v2(x,y)
end

function checkRefract(x1, y1, y2, surface::Surface)
    # we have a line between y1 and y2. If we intersect the surface in that line segment,
    # do the refraction. If we refract, return a new y2 such that the direction is correct for
    # the exact refraction we did out of y1.
    
    intersectfxn = _x -> surface.func(_x + x1, y1 + _x * (y2 - y1) / Δx)
    
    if x1 + Δx < surface.x1 || x1 > surface.x2
        # if the surface isn't in range, just skip it
        return (y2, false)
    end
    
#     print("in range ", x1, " ")
    
    xint = 0
    r = intersectfxn(xint)
    # bound x pretty loosely so if it oscillates a little it doesn't die prematurely
    i = 0
    while abs(r) > 1e-8 && i < 16
        J = ForwardDiff.derivative(intersectfxn, xint)
        xint -= r / J
        r = intersectfxn(xint)
        i = i + 1
    end
    
#     println(i, " ", r, " ", xint, " ")
    
    # if the intersection isn't inside the line segment or we didn't find a good solution, don't refract
    if abs(r) > 1e-8 || xint < 0 || xint > Δx || x1 + xint < surface.x1 || x1 + xint > surface.x2
        return (y2, false)
    end
    
    println("refracting!")
    
    # do the refraction
    xrefr = x1 + xint
    yrefr = y1 + xint * (y2 - y1) / Δx
    v1 = surface.vLeft(xrefr, yrefr)
    v2 = surface.vRight(xrefr, yrefr)
    n = ForwardDiff.gradient(_x->surface.func(_x[1], _x[2]), [xrefr, yrefr])
    n = n ./ norm(n)
    l = [Δx, y2 - y1]
    l = l ./ norm(l)
    lo = snell(l, n, v1, v2)
#     println(l, " ", n, " ", lo)
    
    # if the refraction/reflection leads to a negative x component, we can't keep tracing
    if lo[1] < 0
        return (NaN, true)
    end
    
    # otherwise, go from y1 to a new (x2, y2) in the correct refracted direction
    Δy = lo[2] * Δx / lo[1]
    return (y1 + Δy, true)
end

checkRefract (generic function with 1 method)

In [134]:
surf1 = Surface((_x, _y) -> ((_x .- 5) .^ 2 + (_y) .^ 2) .- 9, 1.5, 2.5, (_x, _y) -> 1.0, (_x, _y) -> 1/1.5)
surf2 = Surface((_x, _y) -> (_x .^ 2 + _y .^ 2) .- 9, 2.5, 3.5, (_x, _y) -> 1/1.5, (_x, _y) -> 1.0)
# surf2 = Surface((_x, _y) -> (_x - 2.99999), 2.5, 3.5, (_x, _y) -> 1.5, (_x, _y) -> 1.0)

ys = zeros(n)
ys[1] = 0.5
ys[2] = 0.5
for k = 2:(n-1)
    x = (k-1) * Δx
    Δy = ys[k] - ys[k-1]
    y2 = ys[k] + Δy
    y2o, _ = checkRefract(x, ys[k], y2, surf1)
    y2o, _ = checkRefract(x, ys[k], y2o, surf2)
    ys[k+1] = y2o
end

contour(LinRange(surf1.x1, surf1.x2, 20), LinRange(-3, 3, 100), surf1.func, levels=-0.1:0.2:0.1, cbar=false)
contour!(LinRange(surf2.x1, surf2.x2, 20), LinRange(-3, 3, 100), surf2.func, levels=-0.1:0.2:0.1, cbar=false)
plot!(LinRange(0,w,n),ys, markershape=:circle, markersize=1, aspect_ratio=:equal, xlim=[0.5, 4.5], ylim=[-2, 2])


refracting!
refracting!


In [135]:
surf1 = Surface((_x, _y) -> (_y.-2), 0, w, (_x, _y) -> 1.0/1.5, (_x, _y) -> 1.0)
surf2 = Surface((_x, _y) -> (_y.+2), 0, w, (_x, _y) -> 1.0/1.5, (_x, _y) -> 1.0)

ys = zeros(n)
ys[1] = 0.0
ys[2] = 0.1
for k = 2:(n-1)
    x = (k-1) * Δx
    Δy = ys[k] - ys[k-1]
    y2 = ys[k] + Δy
    y2o, _ = checkRefract(x, ys[k], y2, surf1)
    y2o, _ = checkRefract(x, ys[k], y2o, surf2)
    ys[k+1] = y2o
end

contour(LinRange(surf1.x1, surf1.x2, 20), LinRange(-3, 3, 100), surf1.func, levels=-0.1:0.2:0.1, cbar=false)
contour!(LinRange(surf2.x1, surf2.x2, 20), LinRange(-3, 3, 100), surf2.func, levels=-0.1:0.2:0.1, cbar=false)
plot!(LinRange(0,w,n),ys)


refracting!
refracting!
refracting!


In [117]:
function v(x, y)
    return (1 .+ abs.(y ./ 4.5) .^ 2)
end


function SSeg(y1, y2)
    return sqrt(Δx ^ 2 + (y2 - y1) ^ 2)
end

function TSeg(y1, y2, x)
    return SSeg(y1, y2) / v(x+Δx/2, (y2+y1)/2)
end

#Discrete Euler-Lagrange Equation
function DEL(ym,yk,yn,x)
    s1 = ForwardDiff.derivative(_y -> TSeg(ym, _y, x), yk)
    s2 = ForwardDiff.derivative(_y -> TSeg(_y, yn, x+Δx), yk)
    return s1 + s2
end

DEL (generic function with 1 method)

In [118]:
ytraj = zeros(n)
ytraj[2] = 0.1

for k = 2:(n-1)
    x = (k-1) * Δx
    ytraj[k+1] = ytraj[k]
    @printf("\n%d: ", k)
    r = DEL(ytraj[k-1],ytraj[k],ytraj[k+1], x)
    while abs(r) > 1e-12
        @printf("%f ", r)
        J = ForwardDiff.derivative(_y -> DEL(ytraj[k-1],ytraj[k],_y, x), ytraj[k+1])
        ytraj[k+1] -= r / J
        r = DEL(ytraj[k-1],ytraj[k],ytraj[k+1], x)
    end
end

#Compare against reference IPOPT solution
xs = LinRange(0, w + 0.1, 100)
ys = LinRange(-1, 1, 100)
plot!(contour(xs, ys, v(xs' .* ones(100), ones(100)' .* ys)))
plot!(LinRange(0,w,n),ytraj)
xlabel!("X")
ylabel!("Y")


2: 0.313928 0.014402 0.000103 0.000000 
3: 0.307817 0.013717 0.000092 0.000000 
4: 0.298742 0.012708 0.000077 0.000000 
5: 0.286860 0.011435 0.000060 0.000000 
6: 0.272366 0.009974 0.000044 0.000000 
7: 0.255484 0.008407 0.000029 0.000000 
8: 0.236457 0.006820 0.000018 0.000000 
9: 0.215538 0.005295 0.000010 0.000000 
10: 0.192976 0.003901 0.000005 0.000000 
11: 0.169017 0.002695 0.000002 0.000000 
12: 0.143893 0.001715 0.000001 
13: 0.117825 0.000975 0.000000 
14: 0.091017 0.000470 0.000000 
15: 0.063660 0.000172 0.000000 
16: 0.035933 0.000036 0.000000 
17: 0.008007 0.000001 
18: -0.019953 -0.000001 
19: -0.047785 -0.000042 -0.000000 
20: -0.075324 -0.000191 -0.000000 
21: -0.102399 -0.000506 -0.000000 
22: -0.128832 -0.001031 -0.000000 
23: -0.154432 -0.001791 -0.000001 
24: -0.178995 -0.002792 -0.000002 -0.000000 
25: -0.202303 -0.004015 -0.000005 -0.000000 
26: -0.224127 -0.005422 -0.000011 -0.000000 
27: -0.244226 -0.006955 -0.000019 -0.000000 
28: -0.262355 -0.008542 -0.000030 

## Putting it all together

In [180]:
function SSeg(y1, y2)
    return sqrt(Δx ^ 2 + (y2 - y1) ^ 2)
end

function TSeg(y1, y2, x, vv)
    return SSeg(y1, y2) / vv(x+Δx/2, (y2+y1)/2)
end

#Discrete Euler-Lagrange Equation
function DEL(ym,yk,yn,x, vv)
    s1 = ForwardDiff.derivative(_y -> TSeg(ym, _y, x, vv), yk)
    s2 = ForwardDiff.derivative(_y -> TSeg(_y, yn, x+Δx, vv), yk)
    return s1 + s2
end

function traceSystem(y1, y2, surfaces)
    curMedium = surfaces[1].vLeft
    ytraj = zeros(n)
    ytraj[1] = y1
    ytraj[2] = y2
    
    for k = 2:(n-1)
        x = (k-1) * Δx
        ytraj[k+1] = ytraj[k]
#         @printf("\n%d: ", k)
        # first, refract based on the current medium
        resid = (_x, cm) -> DEL(ytraj[k-1],ytraj[k],_x, x, cm)
        r = resid(ytraj[k+1], curMedium)
        while abs(r) > 1e-12
#             @printf("%f ", r)
            J = ForwardDiff.derivative(_x -> resid(_x, curMedium), ytraj[k+1])
            ytraj[k+1] -= r / J
            r = resid(ytraj[k+1], curMedium)
        end
        
        # intersect it with the surfaces
        for surf in surfaces
            ytraj[k+1], newSurf = checkRefract(x, ytraj[k], ytraj[k+1], surf)
            if newSurf
                curMedium = surf.vRight
            end
        end
    end
    
    return ytraj
end

function plotSurface!(surface)
    contour!(LinRange(surface.x1, surface.x2, 20), LinRange(-3, 3, 100), surface.func, levels=-0.1:0.2:0.1, cbar=false)
end

plotSurface! (generic function with 1 method)

In [185]:
surfaces = [
    Surface((_x, _y) -> ((_x .- 7) .^ 2 + (_y) .^ 2) .- 25, 1.5, 2.5, (_x, _y) -> 1.0, (_x, _y) -> 1/1.5),
    Surface((_x, _y) -> ((_x .+ 2) .^ 2 + _y .^ 2) .- 25, 2.5, 3.5, (_x, _y) -> 1/1.5, (_x, _y) -> 1.0),
    Surface((_x, _y) -> _x .- 7.5, 7, 8, (_x, _y) -> 1/1.0, (_x, _y) -> (1 .+ abs.(((_x .- 7.5) ./ 40.0 ) .^2 - _y ./ 4.5) .^ 2)),
]

y1 = traceSystem(0.5, 0.5, surfaces)
y2 = traceSystem(0.3, 0.3, surfaces)
y3 = traceSystem(-0.2, -0.2, surfaces)
y4 = traceSystem(-0.4, -0.4, surfaces)
plot()
plotSurface!(surfaces[1])
plotSurface!(surfaces[2])
plotSurface!(surfaces[3])
plot!(LinRange(0,w,n),y1, markershape=:circle, markersize=1, ylim=[-2, 2], legend=false)
plot!(LinRange(0,w,n),y2, markershape=:circle, markersize=1, ylim=[-2, 2], legend=false)
plot!(LinRange(0,w,n),y3, markershape=:circle, markersize=1, ylim=[-2, 2], legend=false)
plot!(LinRange(0,w,n),y4, markershape=:circle, markersize=1, ylim=[-2, 2], legend=false)
contour!(LinRange(7.5, 30, 100), LinRange(-2, 2, 100), surfaces[3].vRight, levels=1.0:0.0025:1.03)

refracting!
refracting!
refracting!
refracting!
refracting!
refracting!
refracting!
refracting!
refracting!
refracting!
refracting!
refracting!


In [186]:
surfaces = [
    Surface((_x, _y) -> ((_x .- 7) .^ 2 + (_y) .^ 2) .- 25, 1.5, 2.5, (_x, _y) -> 1.0, (_x, _y) -> 1/1.5),
    Surface((_x, _y) -> ((_x .+ 2) .^ 2 + _y .^ 2) .- 25, 2.5, 3.5, (_x, _y) -> 1/1.5, (_x, _y) -> 1.0),
]

y1 = traceSystem(0.5, 0.5, surfaces)
y2 = traceSystem(0.3, 0.3, surfaces)
y3 = traceSystem(-0.2, -0.2, surfaces)
y4 = traceSystem(-0.4, -0.4, surfaces)
plot()
plotSurface!(surfaces[1])
plotSurface!(surfaces[2])
plot!(LinRange(0,w,n),y1, markershape=:circle, markersize=1, ylim=[-2, 2], legend=false)
plot!(LinRange(0,w,n),y2, markershape=:circle, markersize=1, ylim=[-2, 2], legend=false)
plot!(LinRange(0,w,n),y3, markershape=:circle, markersize=1, ylim=[-2, 2], legend=false)
plot!(LinRange(0,w,n),y4, markershape=:circle, markersize=1, ylim=[-2, 2], legend=false)


refracting!
refracting!
refracting!
refracting!
refracting!
refracting!
refracting!
refracting!
